# OpenScope Data Extraction

This notebook demonstrates **what data can be extracted from OpenScope** using the project's extraction function.

**Extraction Method:** `extract_optimal_game_state()` from `environment.utils`

This function extracts:
- **14 core features** per aircraft (required by StateProcessor)
- **Additional ATC-critical data** including wind components, flight phase, waypoints, approach status, etc.
- **Global state**: score, time, conflicts

**Purpose**: To see exactly what data is available from OpenScope and how it's structured.

**Prerequisites**:
- OpenScope server running at http://localhost:3003
- Browser automation setup


## Setup and Imports


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import json
import time
from pathlib import Path
from typing import Dict, Any, List
from environment import PlaywrightEnv
from environment.utils import extract_optimal_game_state, execute_command

print("✅ Imports complete")
print("   - PlaywrightEnv for environment setup")
print("   - extract_optimal_game_state() - Production extraction with core features + ATC-critical data")
print("   - execute_command() - Execute commands in OpenScope")


✅ Imports complete
   - PlaywrightEnv for environment setup
   - extract_optimal_game_state() - Production extraction with core features + ATC-critical data
   - execute_command() - Execute commands in OpenScope


## Initialize Environment


In [ ]:
# Create the environment directly (PlaywrightEnv accepts these parameters)
env = PlaywrightEnv(
    airport="KLAS",
    max_aircraft=10,
    episode_length=300,
    headless=False  # Keep visible to see what's happening
)

# Reset to get initial state
obs, info = env.reset()

print("✅ Environment initialized")
print(f"   Aircraft count: {info.get('aircraft_count', 0)}")


✅ Environment initialized
   Aircraft count: 0


## Extract OpenScope Data

Using the project's extraction function to get all available game state data.

**Helper Function**: `print_game_state()` - Displays complete state structure including all aircraft, conflicts, and global data.


### Define Helper Function and Extract Initial Game State


In [ ]:
# Define helper function to print complete game state
def print_game_state(state: Dict[str, Any], title: str = "COMPLETE STATE DATA STRUCTURE"):
    """Print all data from extracted game state in a structured format."""
    print(f"\n{'='*70}")
    print(f"📊 {title}")
    print(f"{'='*70}")
    
    # 1. Top-level state structure
    print(f"\n1️⃣  TOP-LEVEL STATE KEYS:")
    print(f"   All keys in state: {list(state.keys())}")
    print(f"\n   Global State Values:")
    for key in ['score', 'time', 'numAircraft']:
        val = state.get(key)
        print(f"      {key:15s}: {val}")

    # 1b. Scoring events (if available)
    events = state.get('events', [])
    event_counts = state.get('event_counts', {})
    if events or event_counts:
        print(f"\n   Scoring Events:")
        print(f"      events (count this step): {len(events)}")
        if events:
            # Show last up to 10 events
            tail = events[-10:]
            for idx, e in enumerate(tail, 1):
                print(f"         {idx:2d}. {e.get('event', 'N/A')} @ {e.get('ts', 'N/A')}")
        if event_counts:
            print(f"      cumulative event_counts (top 10):")
            # Show up to 10 counts sorted by descending
            items = sorted(event_counts.items(), key=lambda kv: kv[1], reverse=True)[:10]
            for name, cnt in items:
                print(f"         {name:35s}: {cnt}")
    
    # 2. Aircraft data - summary and detailed
    aircraft_list = state.get('aircraft', [])
    print(f"\n2️⃣  AIRCRAFT DATA:")
    print(f"   Total Aircraft: {len(aircraft_list)}")
    print(f"   numAircraft (from state): {state.get('numAircraft', 'N/A')}")
    
    if aircraft_list:
        # Show summary of all aircraft
        print(f"\n   📋 All Aircraft Summary:")
        for i, ac in enumerate(aircraft_list[:10], 1):  # Show first 10
            callsign = ac.get('callsign', 'N/A')
            category = ac.get('category', 'N/A')
            altitude = ac.get('altitude', 'N/A')
            print(f"      {i:2d}. {callsign:10s} | {category:10s} | Alt: {altitude}")
        if len(aircraft_list) > 10:
            print(f"      ... and {len(aircraft_list) - 10} more aircraft")
        
        # Show detailed properties of first aircraft
        sample_ac = aircraft_list[0]
        print(f"\n   ✈️  Detailed Sample Aircraft (Index 0: {sample_ac.get('callsign', 'N/A')}):")
        print(f"      Total Properties: {len(sample_ac)}")
        
        # Group properties logically
        core_14 = ['position', 'altitude', 'heading', 'speed', 'groundSpeed',
                   'assignedAltitude', 'assignedHeading', 'assignedSpeed',
                   'category', 'isOnGround', 'isTaxiing', 'isEstablished', 'targetRunway']
        
        atc_critical = ['windComponents', 'flightPhase', 'nextWaypoint', 'currentWaypoint',
                        'flightPlanAltitude', 'flightPlanRoute', 'hasApproachClearance',
                        'isOnFinal', 'isEstablishedOnGlidepath']
        
        operational = ['isControllable', 'transponderCode', 'groundTrack', 
                       'trueAirspeed', 'climbRate', 'distance']
        
        # Display grouped properties
        def print_group(name, props):
            found = [(p, sample_ac[p]) for p in props if p in sample_ac]
            if found:
                print(f"\n      {name}:")
                for prop, val in found:
                    if isinstance(val, list) and len(val) == 2:
                        print(f"         {prop:25s}: [{val[0]:7.2f}, {val[1]:7.2f}]")
                    elif isinstance(val, dict):
                        if prop == 'windComponents':
                            print(f"         {prop:25s}: head={val.get('head', 'N/A'):6.2f} kts, cross={val.get('cross', 'N/A'):6.2f} kts")
                        else:
                            print(f"         {prop:25s}: {list(val.keys()) if val else '{}'}")
                    elif isinstance(val, (int, float)):
                        print(f"         {prop:25s}: {val}")
                    elif val is not None:
                        print(f"         {prop:25s}: {val}")
        
        print_group("Core 14 Features (for StateProcessor)", core_14)
        print_group("ATC-Critical Data", atc_critical)
        print_group("Operational State", operational)
        
        # Show any remaining properties
        all_shown = set(core_14 + atc_critical + operational + ['callsign'])
        remaining = [(k, v) for k, v in sorted(sample_ac.items()) if k not in all_shown and v is not None]
        if remaining:
            print(f"\n      Additional Properties:")
            for prop, val in remaining:
                if isinstance(val, (int, float)):
                    print(f"         {prop:25s}: {val}")
                elif isinstance(val, list) and len(val) == 2:
                    print(f"         {prop:25s}: [{val[0]:7.2f}, {val[1]:7.2f}]")
                else:
                    print(f"         {prop:25s}: {val}")
    else:
        print("   ⚠️  No aircraft data available")
    
    # 3. Conflicts data - complete
    conflicts_list = state.get('conflicts', [])
    print(f"\n3️⃣  CONFLICTS DATA:")
    print(f"   Total Conflicts: {len(conflicts_list)}")
    
    if conflicts_list:
        print(f"\n   🔍 Conflict Data Structure:")
        sample_conflict = conflicts_list[0]
        print(f"      Properties per conflict: {list(sample_conflict.keys())}")
        
        print(f"\n   📋 All Conflicts:")
        for i, conflict in enumerate(conflicts_list, 1):
            ac1 = conflict.get('aircraft1', 'N/A')
            ac2 = conflict.get('aircraft2', 'N/A')
            distance = conflict.get('distance', 'N/A')
            altitude = conflict.get('altitude', 'N/A')
            has_conflict = conflict.get('hasConflict', 'N/A')
            has_violation = conflict.get('hasViolation', 'N/A')
            print(f"      {i}. {ac1} ↔ {ac2} | Distance: {distance} | Alt: {altitude} | Conflict: {has_conflict} | Violation: {has_violation}")
    else:
        print(f"   ℹ️  No active conflicts")
        print(f"      (Conflict structure available when conflicts exist)")
        print(f"      Properties: aircraft1, aircraft2, distance, altitude, hasConflict, hasViolation")
    
    # 4. Complete data summary
    print(f"\n4️⃣  COMPLETE DATA SUMMARY:")
    print(f"   ✅ Top-level keys: {len(state.keys())} ({', '.join(state.keys())})")
    print(f"   ✅ Aircraft: {len(aircraft_list)} aircraft with {len(sample_ac) if aircraft_list else 0} properties each")
    print(f"   ✅ Conflicts: {len(conflicts_list)} conflicts with {len(conflicts_list[0].keys()) if conflicts_list else 0} properties each")
    print(f"   ✅ Global state: score, time, numAircraft, events, event_counts")
    
    print(f"\n{'='*70}")
    print("✅ All extractable data displayed above")
    print(f"{'='*70}")

# Extract initial game state
page = env.browser_manager.page
state = extract_optimal_game_state(page)
print("✅ Extracted initial game state")
print_game_state(state, "INITIAL STATE DATA STRUCTURE")


✅ Extracted initial game state

📊 INITIAL STATE DATA STRUCTURE

1️⃣  TOP-LEVEL STATE KEYS:
   All keys in state: ['aircraft', 'conflicts', 'score', 'time', 'numAircraft', 'events', 'event_counts']

   Global State Values:
      score          : 0
      time           : 2.832602
      numAircraft    : 14

2️⃣  AIRCRAFT DATA:
   Total Aircraft: 14
   numAircraft (from state): 14

   📋 All Aircraft Summary:
       1. FFT727     | arrival    | Alt: 13000
       2. NKS6307    | arrival    | Alt: 24000
       3. SWA2956    | arrival    | Alt: 11000
       4. SWA9601    | arrival    | Alt: 16000
       5. FFT5443    | arrival    | Alt: 18000
       6. AAL355     | arrival    | Alt: 19000
       7. FFT206     | arrival    | Alt: 23000
       8. ASA27      | arrival    | Alt: 16000
       9. SWA1226    | arrival    | Alt: 21000
      10. FFT193     | arrival    | Alt: 27000
      ... and 4 more aircraft

   ✈️  Detailed Sample Aircraft (Index 0: FFT727):
      Total Properties: 29

      Core 1

In [ ]:
# Set timewarp to 1000x and extract state after 20 seconds
print("⏩ Setting timewarp to 1000x...")
execute_command(page, "timewarp 1000")

print("⏳ Waiting 20 seconds (simulated time: ~5.5 hours at 1000x speed)...")
time.sleep(20)

# Extract state again after timewarp
state_after_timewarp = extract_optimal_game_state(page)
print("✅ Extracted game state after timewarp")
print_game_state(state_after_timewarp, "STATE AFTER 20 SECONDS AT 1000X TIMEWARP")


⏩ Setting timewarp to 1000x...
⏳ Waiting 20 seconds (simulated time: ~5.5 hours at 1000x speed)...
✅ Extracted game state after timewarp

📊 STATE AFTER 20 SECONDS AT 1000X TIMEWARP

1️⃣  TOP-LEVEL STATE KEYS:
   All keys in state: ['aircraft', 'conflicts', 'score', 'time', 'numAircraft', 'events', 'event_counts']

   Global State Values:
      score          : -26130
      time           : 22.824402000000003
      numAircraft    : 30

   Scoring Events:
      events (count this step): 326
          1. DEPARTURE @ 1761796470037
          2. AIRSPACE_BUST @ 1761796470066
          3. DEPARTURE @ 1761796470123
          4. DEPARTURE @ 1761796470181
          5. DEPARTURE @ 1761796470297
          6. DEPARTURE @ 1761796470367
          7. DEPARTURE @ 1761796470399
          8. AIRSPACE_BUST @ 1761796470457
          9. DEPARTURE @ 1761796470458
         10. DEPARTURE @ 1761796470458
      cumulative event_counts (top 10):
         DEPARTURE                          : 189
         AIRSPACE_

In [ ]:
# Test: Verify event hook is working and capturing scoring events
print("=" * 70)
print("EVENT CAPTURE TEST")
print("=" * 70)

# 1. Check if hook is installed
hook_present = page.evaluate("() => typeof window._rlDrainEvents === 'function'")
print(f"\n✓ Hook installed: {hook_present}")

# 2. Check if gameController exists and is wrapped
gc_check = page.evaluate("""() => {
    const gc = window.gameController || window.GameController || (window.app && window.app.gameController);
    if (!gc) return 'GameController not found';
    if (!gc.events_recordNew) return 'events_recordNew method not found';
    if (!gc._eventsRecordNewOriginal) return 'NOT WRAPPED - hook did not attach';
    return 'WRAPPED - hook is active';
}""")
print(f"✓ GameController status: {gc_check}")

# 3. Extract current state and show events
print(f"\n--- Current State ---")
current_state = extract_optimal_game_state(page)
print(f"Score: {current_state.get('score')}")
print(f"Events in buffer: {len(current_state.get('events', []))}")
counts = current_state.get('event_counts', {})
if counts:
    print(f"Event counts: {counts}")
    print(f"\nTop events:")
    for name, cnt in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {name:35s}: {cnt}")
else:
    print("Event counts: {} (empty)")

# 4. Show recent events
events = current_state.get('events', [])
if events:
    print(f"\nRecent events (last 20):")
    for e in events[-20:]:
        print(f"  - {e.get('event')} @ {e.get('ts')}")
else:
    print("\n⚠️  No events captured in this extract")
    print("   This could mean:")
    print("   - No scoring events occurred since last extract (buffer was drained)")
    print("   - Hook attached but events haven't fired yet")
    print("   - Need to induce events (violations, landings, departures)")

print("\n" + "=" * 70)


EVENT CAPTURE TEST

✓ Hook installed: True
✓ GameController status: WRAPPED - hook is active

--- Current State ---
Score: -26710
Events in buffer: 2
Event counts: {'DEPARTURE': 189, 'AIRSPACE_BUST': 138, 'COLLISION': 1}

Top events:
  DEPARTURE                          : 189
  AIRSPACE_BUST                      : 138
  COLLISION                          : 1

Recent events (last 20):
  - AIRSPACE_BUST @ 1761796470470
  - AIRSPACE_BUST @ 1761796470471



In [ ]:
# Display scoring events from extracted states, if available
try:
    def show_events(label, s):
        counts = s.get("event_counts", {}) if isinstance(s, dict) else {}
        events = s.get("events", []) if isinstance(s, dict) else []
        print(f"\n=== {label} ===")
        if counts:
            nz = {k: v for k, v in counts.items() if v}
            print("Non-zero event_counts:", nz)
        else:
            print("No event_counts available")
        if events:
            recent = [e.get("event") for e in events][-20:]
            print("Recent events (last 20):", recent)
        else:
            print("No recent events in this extract")

    if 'state' in globals():
        show_events('Initial state', state)
    if 'state_after_timewarp' in globals():
        show_events('After timewarp', state_after_timewarp)
except Exception as e:
    print("Event display error:", e)



=== Initial state ===
No event_counts available
No recent events in this extract

=== After timewarp ===
Non-zero event_counts: {'DEPARTURE': 189, 'AIRSPACE_BUST': 136, 'COLLISION': 1}
Recent events (last 20): ['DEPARTURE', 'DEPARTURE', 'AIRSPACE_BUST', 'DEPARTURE', 'AIRSPACE_BUST', 'DEPARTURE', 'AIRSPACE_BUST', 'DEPARTURE', 'AIRSPACE_BUST', 'AIRSPACE_BUST', 'DEPARTURE', 'AIRSPACE_BUST', 'DEPARTURE', 'DEPARTURE', 'DEPARTURE', 'DEPARTURE', 'DEPARTURE', 'AIRSPACE_BUST', 'DEPARTURE', 'DEPARTURE']


## Cleanup


In [ ]:
# DIAGNOSTIC: Find where GameController lives
print("=" * 70)
print("SEARCHING FOR GAMECONTROLLER")
print("=" * 70)

search_result = page.evaluate("""() => {
    const results = [];
    
    // 1. Check common locations
    results.push(['window.gameController', typeof window.gameController, !!window.gameController]);
    results.push(['window.GameController', typeof window.GameController, !!window.GameController]);
    results.push(['window.app?.gameController', typeof (window.app && window.app.gameController), !!(window.app && window.app.gameController)]);
    
    // 2. Search for objects with events_recordNew
    let found = [];
    for (const key in window) {
        try {
            const val = window[key];
            if (val && typeof val === 'object' && typeof val.events_recordNew === 'function') {
                found.push({
                    key: key,
                    hasGame: !!val.game,
                    hasEvents: !!val.game?.events,
                    hasScore: val.game?.score !== undefined,
                    score: val.game?.score
                });
            }
        } catch (e) {}
    }
    
    // 3. Check if score element exists (to confirm game is running)
    const scoreEl = document.querySelector('#score');
    const scoreText = scoreEl ? scoreEl.textContent : null;
    
    return {
        commonLocations: results,
        foundControllers: found,
        domScore: scoreText
    };
}""")

print("\n1. Common Locations:")
for loc in search_result['commonLocations']:
    print(f"   {loc[0]:40s} | type: {loc[1]:10s} | exists: {loc[2]}")

print(f"\n2. Objects with events_recordNew method:")
if search_result['foundControllers']:
    for ctrl in search_result['foundControllers']:
        print(f"   window.{ctrl['key']}")
        print(f"      hasGame: {ctrl['hasGame']}, hasEvents: {ctrl['hasEvents']}, score: {ctrl.get('score')}")
else:
    print("   ⚠️  None found")

print(f"\n3. DOM Score element: {search_result['domScore']}")

print("\n" + "=" * 70)


SEARCHING FOR GAMECONTROLLER

1. Common Locations:
   window.gameController                    | type: object     | exists: True
   window.GameController                    | type: undefined  | exists: False
   window.app?.gameController               | type: undefined  | exists: False

2. Objects with events_recordNew method:
   window.__rl_gc_val
      hasGame: True, hasEvents: True, score: -26710

3. DOM Score element: -26710



In [8]:
env.close()
print("✅ Environment closed")


✅ Environment closed
